In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-26 01:56:57--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-10-26 01:56:57 (6.59 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url) 
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [8]:
# Create filtered DF
filter_vine_df = vine_df.filter("total_votes>20")
filter_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [14]:
# 50% of all votes are helpful votes
import pyspark.sql.functions as f
vine_percent_df = filter_vine_df.withColumn('helpful_percent', f.col('helpful_votes')/f.col('total_votes')*100) 
vine50_df = vine_percent_df.filter('helpful_percent>50')
vine50_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|86.20689655172413|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|            100.0|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|93.54838709677419|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|96.29629629629629|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|95.23809523809523|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
only showing top 5 rows



In [19]:
# is vine paid?
paid_vine_df = vine50_df.filter(vine50_df.vine=='Y')
paid_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R3KPC0NBUDASX3|          5|           25|         25|   Y|                N|            100.0|
|R119P2A95GGXX4|          5|           26|         28|   Y|                N|92.85714285714286|
|R1HKIRME8AJ89Z|          5|           79|         82|   Y|                N|96.34146341463415|
|R15KH3FBSVYGBU|          5|           32|         37|   Y|                N|86.48648648648648|
|R22XZEQLVLWW49|          5|           28|         33|   Y|                N|84.84848484848484|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
only showing top 5 rows



In [20]:
# is vine unpaid?
unpaid_vine_df = vine50_df.filter(vine50_df.vine=='N')
unpaid_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|86.20689655172413|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|            100.0|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|93.54838709677419|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|96.29629629629629|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|95.23809523809523|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
only showing top 5 rows



In [30]:
# paid totals
from pyspark.sql.functions import count
#paid_vine_df.groupBy('star_rating').count().show()

total_paid = paid_vine_df.count()
total_paid_5star = paid_vine_df.filter(paid_vine_df.star_rating=='5').count()

55


In [32]:
print("Total Paid: ", total_paid)
print("Total Paid 5*: ", total_paid_5star)
print("% Paid 5*:", total_paid_5star/total_paid * 100)

Total Paid:  103
Total Paid 5*:  55
% Paid 5*: 53.398058252427184


In [33]:
# unpaid totals
total_unpaid = unpaid_vine_df.count()
total_unpaid_5star = unpaid_vine_df.filter(unpaid_vine_df.star_rating=='5').count()

In [34]:
print("Total Unpaid: ", total_unpaid)
print("Total Unpaid 5*: ", total_unpaid_5star)
print("% Unpaid 5*:", total_unpaid_5star/total_unpaid * 100)

Total Unpaid:  37372
Total Unpaid 5*:  19723
% Unpaid 5*: 52.77480466659531
